# Data Cleanup

## Necessary Steps

1. Understand the data structure: Look through the database, understand the main structure, i.e., what each row represents, and what columns and types of information are available
2. Merge game file to add in week identifier to the new dataset (games.csv is in '1. Additional Data File')
3. Create column categorizations to filter dataset for relevant purposes
4. Break down receiver into its individual row
5. Create playmaker column, and check id uniqueness - check that one ID corresponds to one name
6. Flag non-relevant plays - add a binary column that flags 1 for run, pass, reception, FG/XP, 0 for all others
7. Add any additional stat needed - reception (for plays that fall under 'reception'), target ('reception', 'pass')
8. Add position based off of the highest stat of a player. Position will be refined later with web scraping
9. Ensure that stat are correctly represented for a given position
10. Verify top 50 stats against the reported ones


In [55]:
print('u')

## Setup Environment

In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
pd.set_option('display.max_columns', None)

## 1. Upload Data

In [ ]:
'''
Not sure why your folder names were changes. The working directory of this file is where its source code is located
so you can specify directories relatively
'''

In [2]:
# Store your files in the same folder as the source code, so you don't have to specify the directory
df_file_2019 = "../../1. Raw-Data/data2019.csv"
df_file_game = "../1. Additional-Data/games.csv"

# Use the convention df for dataframes
df1 = pd.read_csv(df_file_2019)
df_games = pd.read_csv(df_file_game)

/Users/royaharon/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (168,169,170,171,176,177,180,181,184,185,190,191,192,193,196,197,205,206,207,220,221,222,223,224,225,228,229,233,234,235,240,242,243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
df1

## 2. Add in Weekly Identifier

In [ ]:
'''
In each play row the week of that game will be displayed as a new column.
This will be done through a merge along the game_id column from the game.csv file. 
'''

In [3]:
df = df1.merge(df_games[["game_id", "week"]], on = 'game_id')

df

Unnamed: 0  play_id     game_id home_team away_team posteam  \
0               1       35  2019090500       CHI        GB      GB   
1               2       50  2019090500       CHI        GB      GB   
2               3       71  2019090500       CHI        GB      GB   
3               4       95  2019090500       CHI        GB      GB   
4               5      125  2019090500       CHI        GB      GB   
...           ...      ...         ...       ...       ...     ...   
45541       45542     3955  2019122915       SEA        SF     SEA   
45542       45543     3977  2019122915       SEA        SF     SEA   
45543       45544     3999  2019122915       SEA        SF     SEA   
45544       45545     4080  2019122915       SEA        SF      SF   
45545       45546     4101  2019122915       SEA        SF     NaN   

      posteam_type defteam side_of_field  yardline_100   game_date  \
0             away     CHI           CHI          35.0  2019-09-05   
1             away     CHI            GB          75.0  2019-09-05   
2             away     CHI            GB          75.0  2019-09-05   
3             away     CHI            GB          75.0  2019-09-05   
4             away     CHI            GB          85.0  2019-09-05   
...            ...     ...           ...           ...         ...   
45541         home      SF            SF           5.0  2019-12-29   
45542         home      SF            SF           5.0  2019-12-29   
45543         home      SF            SF           5.0  2019-12-29   
45544         away     SEA            SF          99.0  2019-12-29   
45545          NaN     NaN            SF           NaN  2019-12-29   

       quarter_seconds_remaining  half_seconds_remaining  \
0                          900.0                  1800.0   
1                          900.0                  1800.0   
2                          873.0                  1773.0   
3                          825.0                  1725.0   
4                          795.0                  1695.0   
...                          ...                     ...   
45541                       22.0                    22.0   
45542                       15.0                    15.0   
45543                       12.0                    12.0   
45544                        9.0                     9.0   
45545                        0.0                     0.0   

       game_seconds_remaining game_half  quarter_end  drive  sp  qtr  down  \
0                      3600.0     Half1            0      1   0    1   NaN   
1                      3600.0     Half1            0      1   0    1   1.0   
2                      3573.0     Half1            0      1   0    1   2.0   
3                      3525.0     Half1            0      1   0    1   3.0   
4                      3495.0     Half1            0      1   0    1   4.0   
...                       ...       ...          ...    ...  ..  ...   ...   
45541                    22.0     Half2            0     16   0    4   2.0   
45542                    15.0     Half2            0     16   0    4   3.0   
45543                    12.0     Half2            0     16   0    4   4.0   
45544                     9.0     Half2            0     17   0    4   1.0   
45545                     0.0     Half2            1     17   0    4   NaN   

       goal_to_go   time   yrdln  ydstogo  ydsnet  \
0             0.0  15:00  CHI 35        0     -10   
1             0.0  15:00   GB 25       10     -10   
2             0.0  14:33   GB 25       10     -10   
3             0.0  13:45   GB 25       10     -10   
4             0.0  13:15   GB 15       20     -10   
...           ...    ...     ...      ...     ...   
45541         1.0  00:22    SF 5        5      72   
45542         1.0  00:15    SF 5        5      72   
45543         1.0  00:12    SF 5        5      72   
45544         0.0  00:09    SF 1       10       2   
45545         NaN  00:00    SF 1        0       2   

                              

## 3. Create Categorization Lists

In [ ]:
'''
The columns in the 2019 data file are arranged into categorical lists.
Many of the lists can be automated due to the presence of key words in their title.
'''

### 3.1 Main Lists

In [56]:

key = ['Flag', 'Playmaker_id', 'Playmaker_name', 'play_id', 'game_id', 'home_team', 'away_team', 'week', 'game_date', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'play_type']


pass_play = ['pass_length','pass_location','air_yards']
run_play = ['run_location', 'run_gap']
yard_info = ['yrdln','ydstogo','ydsnet','yards_gained', "fumble_recovery_1_yards", "fumble_recovery_2_yards", "return_yards"]

receiver_info = [col for col in df.columns if 'receiver' in col]
receiver_info += ['yards_after_catch', 'incomplete_pass', 'complete_pass', 'touchdown', 'pass_touchdown', 'fumble_lost', 'Playmaker_id', 'Playmaker_name']


passer_info = [col for col in df.columns if 'passer' in col]


passer_info += ['yards_after_catch', 'incomplete_pass', 'complete_pass', 'touchdown', 'pass_touchdown', 'fumble_lost', 'Playmaker_id', 'Playmaker_name', 'interception']


rusher_info = [col for col in df.columns if 'rusher' in col]
rusher_info += ['touchdown', 'fumble_lost', 'Playmaker_id', 'Playmaker_name']

two_pt_con


xp = ['field_goal_result', 'kick_distance', 'extra_point_result', 'kicker_']



### 3.2 Other Lists

In [57]:

game_time_info = ['quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'time']

gen_play_info = ['drive', 'sp', 'down', 'goal_to_go','desc','play_type','shotgun','no_huddle','qb_dropback','qb_kneel','qb_spike','qb_scramble']

team_info = ["return_team", 'td_team', 'posteam_time', 'defteam_time', 'total_home_score','total_away_score', 'posteam_score_post','defteam_score_post', 'score_differential', "forced_fumble_player_1_team", "forced_fumble_player_2_team", "solo_tackle_1_team", "solo_tackle_2_team", "assist_tackle_1_team", "assist_tackle_2_team", "assist_tackle_3_team", "assist_tackle_4_team", "fumbled_1_team", "fumbled_2_team", "fumble_recovery_1_team", "fumble_recovery_2_team"]

game_info = ['play_id','game_id','home_team','away_team','posteam','posteam_type', 'defteam', 'side_of_field', 'yardline_100','game_date', "year"]


prob_cols = [col for col in df.columns if 'prob' in col]

penalty_info = [col for col in df.columns if 'penalty' in col]


kickoff_punt_info = [col for col in df.columns if 'kickoff' in col]

kickoff_punt_info += [col for col in df.columns if 'kicker' in col]
kickoff_punt_info += [col for col in df.columns if 'punt' in col]


defensive_info = [col for col in df.columns if 'defensive' in col]

defensive_info += [col for col in df.columns if 'fumble' in col]
defensive_info += [col for col in df.columns if 'sack' in col]
defensive_info += [col for col in df.columns if 'interception' in col]
defensive_info += [col for col in df.columns if 'defense' in col]

epa_info = [col for col in df.columns if 'epa' in col]

wpa_info = [col for col in df.columns if 'wpa' in col]

touchdown_info = [col for col in df.columns if 'touchdown' in col]

timeout_info = [col for col in df.columns if 'timeout' in col]

wp_info = [col for col in df.columns if 'wp' in col]

score_info = [col for col in df.columns if 'score' in col]

tackle_info = [col for col in df.columns if 'tackle' in col]

all_lists = [game_info, team_info, game_time_info ,xp, wpa_info, epa_info, touchdown_info, yard_info, run_play, pass_play, key, prob_cols, penalty_info, kickoff_punt_info, defensive_info, timeout_info, passer_info, receiver_info, rusher_info, score_info, tackle_info]

big_list = [item for elem in all_lists for item in elem]

remainder_list = [col for col in df.columns if col not in big_list]

['passer_player_id',
 'passer_player_name',
 'interception',
 'interception_player_id',
 'interception_player_name',
 'lateral_interception_player_id',
 'lateral_interception_player_name',
 'yards_after_catch',
 'incomplete_pass',
 'complete_pass',
 'touchdown',
 'pass_touchdown',
 'fumble']

## 4. Receiver row creation

In [ ]:
'''
For each pass play, creat an additional row for the recepetion
so that there can be two play makers (the passer and the receiver).
'''

In [25]:
new_df = pd.DataFrame()
new_df = df[(df['play_type']=="pass")]

new_df['play_type'].replace({'pass': 'reception'}, inplace=True)

df = pd.concat([df, new_df], sort= True)

df

Flag  Target  Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
0         0       0           1       NaN       NaN        NaN            0.0   
1         1       0           2       NaN       NaN        NaN            0.0   
2         1       1           3 -1.095212 -0.031647       -1.0            0.0   
3         1       1           4       NaN       NaN        NaN            0.0   
4         0       0           5       NaN       NaN        NaN            0.0   
...     ...     ...         ...       ...       ...        ...            ...   
45535     1       1       45536  4.335141  0.657133       12.0            0.0   
45538     1       1       45539  3.532878  0.610825       11.0            0.0   
45541     1       1       45542  2.888636  0.702366        5.0            0.0   
45542     1       1       45543  3.550080  0.899287        5.0            0.0   
45543     1       1       45544 -2.636491  0.010686        4.0            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
0                      NaN                       NaN   
1                      NaN                       NaN   
2                      NaN                       NaN   
3                      NaN                       NaN   
4                      NaN                       NaN   
...                    ...                       ...   
45535                  NaN                       NaN   
45538                  NaN                       NaN   
45541                  NaN                       NaN   
45542                  NaN                       NaN   
45543                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
0                             NaN                  NaN   
1                             NaN                  NaN   
2                             NaN                  NaN   
3                             NaN                  NaN   
4                             NaN                  NaN   
...                           ...                  ...   
45535                         NaN                  NaN   
45538                         NaN                  NaN   
45541                         NaN                  NaN   
45542                         NaN                  NaN   
45543                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_3_team assist_tackle_4_player_id  \
0                      NaN                       NaN   
1                      NaN            

## 5. Playmaker Column

In [ ]:
'''
Step 5 creates a playmaker column, and also checks if each player 
id matchs only one name and corrects those that do not.

Steven's approach starts by establishing a playmaker id and name column and uses these columns to identify 
which ids have more than one name.

Using grouby you can cross check the playmake_id column and the playmaker_name column and isolate that id that have 
more than one name.

'''

#### 5.1 Define Play Maker

In [27]:

df['play_maker_id'] = np.where(df['play_type']=='run',df['rusher_player_id'], np.NaN)
df['play_maker_id'] = np.where(df['play_type']=='pass', df['passer_player_id'], df['play_maker'])
df['play_maker_id'] = np.where(df['play_type']=='reception',df['receiver_player_id'],df['play_maker'])
df['play_maker_id'] = np.where((df['play_type']=='extra_point')|(df['play_type']=='field_goal'),df['kicker_player_id'],df['play_maker'])

df[df['play_type'] == 'run']

Flag  Target  Unnamed: 0  air_epa  air_wpa  air_yards  assist_tackle  \
1         1       0           2      NaN      NaN        NaN            0.0   
6         1       0           7      NaN      NaN        NaN            1.0   
8         1       0           9      NaN      NaN        NaN            0.0   
10        1       0          11      NaN      NaN        NaN            0.0   
14        1       0          15      NaN      NaN        NaN            0.0   
...     ...     ...         ...      ...      ...        ...            ...   
45513     1       0       45514      NaN      NaN        NaN            0.0   
45515     1       0       45516      NaN      NaN        NaN            0.0   
45519     1       0       45520      NaN      NaN        NaN            0.0   
45520     1       0       45521      NaN      NaN        NaN            0.0   
45544     1       0       45545      NaN      NaN        NaN            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
1                           NaN                         NaN   
6                    00-0033120                     K.Clark   
8                           NaN                         NaN   
10                          NaN                         NaN   
14                          NaN                         NaN   
...                         ...                         ...   
45513                       NaN                         NaN   
45515                       NaN                         NaN   
45519                       NaN                         NaN   
45520                       NaN                         NaN   
45544                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
1                      NaN                       NaN   
6                       GB                00-0032412   
8                      NaN                       NaN   
10                     NaN                       NaN   
14                     NaN                       NaN   
...                    ...                       ...   
45513                  NaN                       NaN   
45515                  NaN                       NaN   
45519                  NaN                       NaN   
45520                  NaN                       NaN   
45544                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
1                             NaN                  NaN   
6                         D.Lowry                   GB   
8                             NaN                  NaN   
10                            NaN                  NaN   
14                            NaN                  NaN   
...                           ...                  ...   
45513                         NaN                  NaN   
45515                         NaN                  NaN   
45519                         NaN                  NaN   
45520                         NaN                  NaN   
45544                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
1                           NaN                         NaN   
6                           NaN                         NaN   
8                           NaN                         NaN   
10                          NaN                         NaN   
14                          NaN                         NaN   
...                         ...                         ...   
45513                       NaN                         NaN   
45515                       NaN                         NaN   
45519                       NaN                         NaN   
45520                       NaN                         NaN   
45544                       NaN                         NaN   

      assist_tackle_3_team assist_tackle_4_player_id  \
1                      NaN                       NaN   
6                      NaN                       NaN   
8     

In [ ]:
# Add in corresponding play maker ID
df['play_maker'] = np.where(df['play_type']=='run', df['rusher_player_name'], np.NaN)
df['play_maker'] = np.where(df['play_type']=='pass',df['passer_player_name'], df['play_maker'])
df['play_maker'] = np.where((df['play_type']=='no_play') & (df['interception']== 1), df['passer_player_name'], df['play_maker'])
df['play_maker'] = np.where(df['play_type']=='reception',df['receiver_player_name'],df['play_maker'])
df['play_maker'] = np.where((df['play_type']=='extra_point')|(df['play_type']=='field_goal'),df['kicker_player_name'],df['play_maker'])
df
# Now that we have a single column to identify play makers, it is a lot easier to check for ID uniqueness

In [14]:
new_df = df.copy()

new_df

Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
0               1       NaN       NaN        NaN            0.0   
1               2       NaN       NaN        NaN            0.0   
2               3 -1.095212 -0.031647       -1.0            0.0   
3               4       NaN       NaN        NaN            0.0   
4               5       NaN       NaN        NaN            0.0   
...           ...       ...       ...        ...            ...   
45535       45536  4.335141  0.657133       12.0            0.0   
45538       45539  3.532878  0.610825       11.0            0.0   
45541       45542  2.888636  0.702366        5.0            0.0   
45542       45543  3.550080  0.899287        5.0            0.0   
45543       45544 -2.636491  0.010686        4.0            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
0                      NaN                       NaN   
1                      NaN                       NaN   
2                      NaN                       NaN   
3                      NaN                       NaN   
4                      NaN                       NaN   
...                    ...                       ...   
45535                  NaN                       NaN   
45538                  NaN                       NaN   
45541                  NaN                       NaN   
45542                  NaN                       NaN   
45543                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
0                             NaN                  NaN   
1                             NaN                  NaN   
2                             NaN                  NaN   
3                             NaN                  NaN   
4                             NaN                  NaN   
...                           ...                  ...   
45535                         NaN                  NaN   
45538                         NaN                  NaN   
45541                         NaN                  NaN   
45542                         NaN                  NaN   
45543                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_3_team assist_tackle_4_player_id  \
0                      NaN                       NaN   
1                      NaN                       NaN   
2                      NaN                       NaN   
3                      NaN                       NaN   
4                      NaN            

#### 5.2 Identify Non Unique Player Names

In [7]:
# Create a data frame that contains the unique count of each player name under a given ID
# Filter on the IDs that correspond to more than one name

nunique_id = df[df.groupby(['play_maker_id'])['play_maker'].transform('nunique') > 1]['play_maker_id'].unique()


In [8]:
# Identify all the duplicate names

df[df['play_maker_id'].isin(nunique_id)]['play_maker'].unique()

array([], dtype=object)

In [9]:
# Create a dictionary of what the corrected names should be

name_corrections = {'D.Chark Jr.': 'D.Chark',
'Jos.Allen':'J.Allen',
'M.Ingram II': 'M.Ingram',
'A.Levine Sr.': 'A.Levine',
'R.Griffin III': 'R.Griffin',
'G.Minshew II':'G.Minshew',
'B.Snell Jr.':'B.Snell', 
'Tr.Edmunds':'T.Edmunds',
'R.James Jr.': 'R.James',
'J.Ross III':'J.Ross',
'W.Snead IV':'W.Snead', 
'M.Jones Jr.': 'M.Jones', 
'M.Sanu Sr.':'M.Sanu', 
'O.Beckham Jr.':'O.Beckham', 
'P.Dorsett II':'P.Dorsett'}

#### S5.3 Correct Name Uniqueness

In [10]:
# Create a function to correct the typos

def typo_correction(name):
    if name in name_corrections.keys():
        return name_corrections[name]
    else:
        return name

In [11]:
# apply the function to the dataframe

df['play_maker'] = df['play_maker'].apply(typo_correction)

#### S5.4 New Attempt

In [12]:
test = df[df['passer_player_id'].isin(nunique_id)].groupby(['passer_player_id','passer_player_name']).size().reset_index()

test.set_index('passer_player_id', inplace=True)

test


Empty DataFrame
Columns: [passer_player_name, 0]
Index: []

In [13]:

d = {player_id:(test.loc[player_id,'passer_player_name'][0],test.loc[player_id,'passer_player_name'][1]) for player_id in nunique_id}


In [14]:
test2 = df.copy()



In [15]:
test2.set_index('passer_player_id', inplace=True)


In [16]:
for dup in d.keys():
    test2.loc[dup,'passer_player_name'] = d[dup][0]

## 6. Relevant Plays

In [82]:
'''
Using a where statement use a binary indicator to Flag relevant plays 
such as passes, receptions, runs, field goals, extra points, and qb_kneels. 
'''

'\nUsing a where statement use a binary indicator to Flag relevant plays \nsuch as passes, receptions, runs, field goals, extra points, and qb_kneels. \n'

In [28]:
df['Flag'] = np.where((df['play_type'] == 'pass') | (df['play_type'] == 'run') | (df['play_type'] == 'field_goal') | (df['play_type'] == 'qb_kneel') | (df['play_type'] == 'extra_point') , 1, 0)


## 7. Extra Columns

In [29]:

df['Target'] = np.where(((df['play_type'] == 'pass') | (df['play_type'] == 'reception')) & (df['yards_after_catch'] != 'nan'), 1, 0)


df['reception'] = np.where(((df['play_type'] == 'pass') | (df['play_type'] == 'reception')) & (df['yards_after_catch'] >= 0), 1, 0)


## 8. Player Positions

In [19]:
'''
In this step positions will be assigned to each player based on the where their greatest number of yardage is focused
in. After their positions are set all the player data will be merged into one table that displays each players yard 
stats across rushing, passing, and receiving yards.
'''

'\nIn this step positions will be assigned to each player based on the where their greatest number of yardage is focused\nin. After their positions are set all the player data will be merged into one table that displays each players yard \nstats across rushing, passing, and receiving yards.\n'

In [83]:
df

Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
0               1       NaN       NaN        NaN            0.0   
1               2       NaN       NaN        NaN            0.0   
2               3 -1.095212 -0.031647       -1.0            0.0   
3               4       NaN       NaN        NaN            0.0   
4               5       NaN       NaN        NaN            0.0   
...           ...       ...       ...        ...            ...   
45535       45536  4.335141  0.657133       12.0            0.0   
45538       45539  3.532878  0.610825       11.0            0.0   
45541       45542  2.888636  0.702366        5.0            0.0   
45542       45543  3.550080  0.899287        5.0            0.0   
45543       45544 -2.636491  0.010686        4.0            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
0                      NaN                       NaN   
1                      NaN                       NaN   
2                      NaN                       NaN   
3                      NaN                       NaN   
4                      NaN                       NaN   
...                    ...                       ...   
45535                  NaN                       NaN   
45538                  NaN                       NaN   
45541                  NaN                       NaN   
45542                  NaN                       NaN   
45543                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
0                             NaN                  NaN   
1                             NaN                  NaN   
2                             NaN                  NaN   
3                             NaN                  NaN   
4                             NaN                  NaN   
...                           ...                  ...   
45535                         NaN                  NaN   
45538                         NaN                  NaN   
45541                         NaN                  NaN   
45542                         NaN                  NaN   
45543                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
0                           NaN                         NaN   
1                           NaN                         NaN   
2                           NaN                         NaN   
3                           NaN                         NaN   
4                           NaN                         NaN   
...                         ...                         ...   
45535                       NaN                         NaN   
45538                       NaN                         NaN   
45541                       NaN                         NaN   
45542                       NaN                         NaN   
45543                       NaN                         NaN   

      assist_tackle_3_team assist_tackle_4_player_id  \
0                      NaN                       NaN   
1                      NaN                       NaN   
2                      NaN                       NaN   
3                      NaN                       NaN   
4                      NaN            

In [9]:
df['playmaker_team'] = df['posteam']

pass_player_team = df[df.play_type == 'pass'][['play_maker', 'playmaker_team']]

pass_player_team = pass_player_team.drop_duplicates()

pass_play = df[df.play_type == 'pass'][['play_maker', 'yards_gained']]

pass_yards = pass_play.groupby('play_maker',as_index = False).yards_gained.sum()

pass_yards = pass_yards.rename(columns = {'yards_gained': 'pass_yards'})

pass_yards_final = pass_yards.merge(pass_player_team[["play_maker", "playmaker_team"]], on = 'play_maker', how = 'outer')

pass_yards_final


play_maker  pass_yards playmaker_team
0        A.Beck           6            DEN
1      A.Dalton        3247            CIN
2    A.Erickson          26            CIN
3      A.Kamara          13             NO
4         A.Lee          26            ARI
..          ...         ...            ...
106      T.Hill          46             NO
107   T.Siemian         -14            NYJ
108    T.Taylor          33            LAC
109     W.Grier         184            CAR
110    Z.Pascal           0            IND

[111 rows x 3 columns]

In [10]:
receiver_player_team = df[df.play_type == 'reception'][['play_maker', 'playmaker_team']]

receiver_player_team = receiver_player_team.drop_duplicates()

reception_play = df[df.play_type == 'reception'][['play_maker', 'yards_gained']]

reception_yards = reception_play.groupby('play_maker', as_index = False).yards_gained.sum()

reception_yards = reception_yards.rename(columns = {'yards_gained': 'reception_yards'})

reception_yards_final = reception_yards.merge(receiver_player_team[["play_maker", "playmaker_team"]], on = 'play_maker', how = 'outer')

reception_yards_final = reception_yards_final.dropna() 

reception_yards

play_maker  reception_yards
0    A.Abdullah               88
1       A.Armah                6
2     A.Auclair               11
3        A.Beck               90
4       A.Brown             1107
..          ...              ...
487    Z.Gentry                4
488     Z.Jones              216
489      Z.Line               36
490    Z.Pascal              609
491    Z.Zenner               19

[492 rows x 2 columns]

In [12]:
rusher_player_team = df[df.play_type == 'run'][['play_maker', 'playmaker_team']]

rusher_player_team = rusher_player_team.drop_duplicates()

run_play = df[df.play_type == 'run'][['play_maker', 'yards_gained']]

run_yards = run_play.groupby('play_maker', as_index = False).yards_gained.sum()

run_yards = run_yards.rename(columns = {'yards_gained': 'run_yards'})

run_yards_final = run_yards.merge(rusher_player_team[["play_maker", "playmaker_team"]], on = 'play_maker', how = 'outer')

run_yards_final = run_yards_final.dropna() 

run_yards_final

play_maker  run_yards playmaker_team
0    A.Abdullah        115            MIN
1       A.Armah         11            CAR
2        A.Beck          3            DEN
3       A.Brown         65             NE
4       A.Brown         65            TEN
..          ...        ...            ...
336     Z.Jones          3            OAK
337      Z.Line         20             NO
338    Z.Pascal         16            IND
339    Z.Zenner         -2             NO
340    Z.Zenner         -2            ARI

[341 rows x 3 columns]

In [13]:
merged = pass_yards_final.merge(reception_yards_final[["play_maker", "reception_yards"]], on = 'play_maker', how = 'outer')
final_merged = merged.merge(run_yards_final[["play_maker", "run_yards"]], on = 'play_maker', how = 'outer')
final_merged['reception_yards'] = final_merged['reception_yards'].fillna(0)
final_merged['pass_yards'] = final_merged['pass_yards'].fillna(0)
final_merged['run_yards'] = final_merged['run_yards'].fillna(0)
final_merged['position'] = np.where((final_merged['pass_yards'] > final_merged['reception_yards']), 'QB', 'WR/TE')
final_merged['position'] = np.where((final_merged['run_yards'] > final_merged['pass_yards']) & (final_merged['run_yards'] > final_merged['reception_yards']) , 'RB', final_merged['position'])
# final_merged
final_merged[final_merged['play_maker'] == 'S.Sims']

play_maker  pass_yards playmaker_team  reception_yards  run_yards position
107     S.Sims         0.0            WAS            310.0       85.0    WR/TE

## 9. Stat Verification

In [ ]:
'''
In this step, the data that yardage data that had been calculated based on the plays in 2019 will be cross checked with
verified data to ensure that the data calculated in step 8 is the same as the verified data.
'''

In [30]:
# df_file_game = "../1. Additional-Data/2019 stats.xlsx"

xls = pd.ExcelFile("../1. Additional-Data/2019 stats.xlsx")

df_WR = pd.read_excel(xls, 'WR')

df_QB = pd.read_excel(xls, 'QB')

df_RB = pd.read_excel(xls, 'RB')


In [ ]:
"""
Reformats the names of the players so that the names in the
validated data file match those in the 2019 data excel sheet
"""

In [31]:
WR_ls1 = [wr[:wr.find('\\')] for wr in df_WR['Player']]
WR_final = []

for wr in WR_ls1:
    if '*' in wr:
        WR_final.append(wr[:wr.find('*')])
    else:
        WR_final.append(wr)

WR_final = [(col[0]+'.'+col[col.find(' '):]).replace(" ", "") for col in WR_final]

df_WR['Player'] = WR_final


In [32]:
RB_ls1 = [rb[:rb.find('\\')] for rb in df_RB['Player']]
RB_final = []

for rb in RB_ls1:
    if '*' in rb:
        RB_final.append(rb[:rb.find('*')])
    else:
        RB_final.append(rb)

RB_final = [(col[0]+'.'+col[col.find(' '):]).replace(" ", "") for col in RB_final]

df_RB['Player'] = RB_final

In [33]:
QB_ls1 = [qb[:qb.find('\\')] for qb in df_QB['Player']]
QB_final = []

for qb in QB_ls1:
    if '*' in qb:
        QB_final.append(qb[:qb.find('*')])
    else:
        QB_final.append(qb)
        

QB_final = [(col[0]+'.'+col[col.find(' '):]).replace(" ", "") for col in QB_final]

df_QB['Player'] = QB_final

In [ ]:
"""
QB stats
"""

In [34]:
# calculates the number of plays each player was the playmaker for
comp_list = df[(df['play_type'] == 'pass') & (df['reception'] == 1) & (df['sack'] != 1)].groupby('play_maker', as_index = False).play_maker_id.count()

Att_list = df[(df['play_type'] == 'pass') & (df['sack'] != 1)].groupby('play_maker', as_index = False).play_maker_id.count()
# comp_list[freq_list['play_maker'] == 'D.Prescott']
# freq_list.head()
# df.play_type.unique()
# df[df['play_type'] == 'Pass']
Att_comp = comp_list.merge(Att_list, on = 'play_maker', how = 'inner')
Att_comp

play_maker  play_maker_id_x  play_maker_id_y
0          A.Beck                1                1
1        A.Dalton              307              518
2      A.Erickson                1                1
3        A.Kamara                1                1
4           A.Lee                1                1
..            ...              ...              ...
88  T.Bridgewater              132              195
89         T.Hill                3                6
90      T.Siemian                3                6
91       T.Taylor                4                6
92        W.Grier               28               52

[93 rows x 3 columns]

In [35]:
pd.set_option('display.max_rows',None)

A = df[(df['play_type'] == 'pass') & (df['sack'] != 1)].groupby('play_maker', as_index = False).agg({
    'yards_gained':['sum', 'max'],
    'pass_touchdown':['sum'],
    'interception':['sum']
    
})

A.columns = ['play_maker', 'Yards', 'Long', 'pass_touchdown', 'interceptions']

A.sort_values(['pass_touchdown'], ascending = False)

play_maker  Yards  Long  pass_touchdown  interceptions
68          L.Jackson   3127    83            36.0            6.0
59          J.Winston   5113    71            33.0           28.0
92           R.Wilson   4112    60            31.0            5.0
36         D.Prescott   4904    62            30.0           11.0
27            D.Brees   2979    61            27.0            4.0
50        J.Garoppolo   3982    75            27.0           12.0
24            C.Wentz   4041    53            27.0            6.0
63          K.Cousins   3605    66            26.0            6.0
83          P.Mahomes   4033    83            26.0            5.0
37           D.Watson   3854    54            26.0           11.0
76             M.Ryan   4468    93            26.0           12.0
6           A.Rodgers   4006    74            26.0            4.0
33            D.Jones   3031    75            24.0           12.0
99            T.Brady   4059    59            24.0            6.0
84           P.Rivers   4617    84            23.0           20.0
51             J.Goff   4638    66            22.0           15.0
91        R.Tannehill   2742    91            22.0            5.0
14         B.Mayfield   3827    89            22.0           20.0
28             D.Carr   4054    75            21.0            8.0
88      R.Fitzpatrick   3531    51            20.0           12.0
65           K.Murray   3728    88            20.0           11.0
78         M.Stafford   2499    66            19.0            5.0
93          S.Darnold   3030    92            19.0           13.0
44         J.Brissett   2944    50            18.0            6.0
61            K.Allen   3322    52            17.0           16.0
79         M.Trubisky   3140    53            17.0           10.0
1            A.Dalton   3496    66            16.0           12.0
43            J.Allen   2411    53            16.0            8.0
75          M.Rudolph   1765    76            13.0            9.0
42       G.Minshew II   1835    70            12.0            4.0
18           C.Keenum   1707    69            11.0            5.0
41          G.Minshew   1442    69             9.0            2.0
100     T.Bridgewater   1384    45             9.0            2.0
30          D.Haskins   1367    75             7.0            7.0
34             D.Lock   1020    48             7.0            3.0
72          M.Mariota   1203    75             7.0            1.0
49           J.Flacco   1824    70             6.0            5.0
38          E.Manning   1042    55             6.0            5.0
31           D.Hodges   1063    79             5.0            8.0
60          Jos.Allen    682    49             4.0            0.0
46          J.Driskel    685    47             4.0            4.0
73            M.Moore    659    57             4.0            0.0
11            B.Hoyer    372    23             4.0            4.0
26           D.Blough    984    75             4.0            4.0
17           C.Daniel    435    37             3.0            2.0
9             B.Allen    515    75             3.0            2.0
77           M.Schaub    582    35             3.0            1.0
81            N.Foles    738    39             3.0            2.0
87           R.Finley    474    47             2.0            2.0
39          E.Sanders     35    35             1.0            0.0
103          T.Taylor     33    14             1.0            0.0
55            J.Rosen    567    40             1.0            5.0
70          M.Glennon     56    19             1.0            0.0
89          R.Griffin     73    13             1.0            0.0
25         D.Amendola     19    19             1.0            0.0
32          D.Hopkins      6     6             1.0            1.0
71            M.Haack      1     1             1.0            0.0
47          J.Edelman     47    32             1.0            0.0
45            J.Brown     28    28             1.0            0.0
98            T.Boyle     15     7             0.0 

In [33]:
df.sack.unique()

array([ 0.,  1., nan])

In [36]:
Final_QB_Stats = A.merge(Att_comp, on = 'play_maker', how = 'inner')
Final_QB_Stats = Final_QB_Stats.rename(columns = {'play_maker_id_x': 'completions'})
Final_QB_Stats = Final_QB_Stats.rename(columns = {'play_maker_id_y': 'attempts'})
# Final_QB_Stats.sort_values(['pass_touchdown'], ascending = False)


Final_QB_Stats

play_maker  Yards  Long  pass_touchdown  interceptions  completions  \
0             A.Beck      6     6             0.0            0.0            1   
1           A.Dalton   3496    66            16.0           12.0          307   
2         A.Erickson     26    26             0.0            0.0            1   
3           A.Kamara     13    13             0.0            0.0            1   
4              A.Lee     26    26             0.0            0.0            1   
5         A.McCarron    225    21             0.0            1.0           21   
6          A.Rodgers   4006    74            26.0            4.0          348   
7           A.Tanney      1     1             0.0            0.0            1   
8           A.Wilson     20    20             0.0            0.0            1   
9            B.Allen    515    75             3.0            2.0           38   
10         B.Bortles      3     3             0.0            0.0            1   
11           B.Hoyer    372    23             4.0            4.0           34   
12         B.Hundley     49    28             0.0            0.0            5   
13            B.Kern     11    11             0.0            0.0            1   
14        B.Mayfield   3827    89            22.0           20.0          311   
15  B.Roethlisberger    351    45             0.0            1.0           35   
16          C.Daniel    435    37             3.0            2.0           44   
17          C.Keenum   1707    69            11.0            5.0          157   
18           C.McCoy    122    22             0.0            1.0           17   
19          C.Newton    572    44             0.0            0.0           50   
20          C.Sutton     38    38             0.0            0.0            1   
21           C.Wentz   4041    53            27.0            6.0          383   
22        D.Amendola     19    19             1.0            0.0            1   
23          D.Blough    984    75             4.0            4.0           93   
24           D.Brees   2979    61            27.0            4.0          280   
25            D.Carr   4054    75            21.0            8.0          358   
26         D.Haskins   1367    75             7.0            7.0          118   
27          D.Hodges   1063    79             5.0            8.0           96   
28         D.Hopkins      6     6             1.0            1.0            1   
29           D.Jones   3031    75            24.0           12.0          277   
30            D.Lock   1020    48             7.0            3.0           98   
31          D.Pettis     16    16             0.0            0.0            1   
32        D.Prescott   4904    62            30.0           11.0          384   
33          D.Watson   3854    54            26.0           11.0          327   
34         E.Manning   1042    55             6.0            5.0           90   
35         E.Sanders     35    35             1.0            0.0            1   
36         G.Minshew   1442    69             9.0            2.0          123   
37      G.Minshew II   1835    70            12.0            4.0          159   
38           J.Allen   2411    53            16.0            8.0          212   
39        J.Brissett   2944    50            18.0            6.0          271   
40           J.Brown     28    28             1.0            0.0            1   
41         J.Driskel    685    47             4.0            4.0           60   
42         J.Edelman     47    32             1.0            0.0            2   
43          J.Flacco   1824    70             6.0            5.0          167   
44       J.Garoppolo   3982    75            27.0           12.0          328   
45            J.Goff   4638    66            22.0           15.0          390   
46          J.Hekker     28    23             0.0            1.0            2   
47          J.McCown     24    13             0.0            0.0            3   
48           J.Rosen    567    40        

In [37]:
df_QB = df_QB.rename(columns = {'Player': 'play_maker'})

valid_table_QB = Final_QB_Stats[['play_maker', 'Yards', 'pass_touchdown', 
                                'interceptions', 'completions', 'attempts']].merge(df_QB[['play_maker', 'Yds', 'TD', 'Int', 'Cmp', 'Att']], on = 'play_maker', how = 'left')

valid_table_QB['delta_yds'] = valid_table_QB['Yards'] - valid_table_QB['Yds']
valid_table_QB['delta_TD'] = valid_table_QB['pass_touchdown'] - valid_table_QB['TD']
valid_table_QB['delta_int'] = valid_table_QB['interceptions'] - valid_table_QB['Int']
valid_table_QB['delta_cmp'] = valid_table_QB['completions'] - valid_table_QB['Cmp']
valid_table_QB['delta_att'] = valid_table_QB['attempts'] - valid_table_QB['Att']
valid_table_QB.sort_values(['Yards'], ascending = False)


KeyError: "['play_maker'] not in index"

In [38]:
J_win = df[(df['play_maker'] == 'J.Winston')]
J_stats = J_win.groupby('game_id', as_index = False).agg({
    'interception':['sum'],
    
})

# 14 and 12 missing ints

J_stats

game_id interception
                        sum
0   2019090811          3.0
1   2019091200          0.0
2   2019092209          1.0
3   2019092909          1.0
4   2019100603          0.0
5   2019101300          5.0
6   2019102708          2.0
7   2019110308          0.0
8   2019111006          2.0
9   2019111706          4.0
10  2019112400          2.0
11  2019120104          0.0
12  2019120808          3.0
13  2019121502          1.0
14  2019122100          4.0
15  2019122912          2.0

In [39]:
df[(df['game_id'] == 2019122100) & (df['interception'] == 1)]
# play_type is set to no_play so there is no play_maker and thus interception is not counted

Flag  Target  Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
39848     1       1       39849  1.763625  0.051776       10.0            0.0   
39853     0       0       39854       NaN       NaN       11.0            0.0   
39891     1       1       39892  0.551639  0.017262       17.0            0.0   
39932     1       1       39933  2.003530  0.056749       42.0            0.0   
40019     1       1       40020  0.666964  0.058817        3.0            0.0   
39848     0       1       39849  1.763625  0.051776       10.0            0.0   
39891     0       1       39892  0.551639  0.017262       17.0            0.0   
39932     0       1       39933  2.003530  0.056749       42.0            0.0   
40019     0       1       40020  0.666964  0.058817        3.0            0.0   
39848     0       1       39849  1.763625  0.051776       10.0            0.0   
39891     0       1       39892  0.551639  0.017262       17.0            0.0   
39932     0       1       39933  2.003530  0.056749       42.0            0.0   
40019     0       1       40020  0.666964  0.058817        3.0            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
39848                       NaN                         NaN   
39853                       NaN                         NaN   
39891                       NaN                         NaN   
39932                       NaN                         NaN   
40019                       NaN                         NaN   
39848                       NaN                         NaN   
39891                       NaN                         NaN   
39932                       NaN                         NaN   
40019                       NaN                         NaN   
39848                       NaN                         NaN   
39891                       NaN                         NaN   
39932                       NaN                         NaN   
40019                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
39848                  NaN                       NaN   
39853                  NaN                       NaN   
39891                  NaN                       NaN   
39932                  NaN                       NaN   
40019                  NaN                       NaN   
39848                  NaN                       NaN   
39891                  NaN                       NaN   
39932                  NaN                       NaN   
40019                  NaN                       NaN   
39848                  NaN                       NaN   
39891                  NaN                       NaN   
39932                  NaN                       NaN   
40019                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
39848                         NaN                  NaN   
39853                         NaN                  NaN   
39891                         NaN                  NaN   
39932                         NaN                  NaN   
40019                         NaN                  NaN   
39848                         NaN                  NaN   
39891                         NaN                  NaN   
39932                         NaN                  NaN   
40019                         NaN                  NaN   
39848                         NaN                  NaN   
39891                         NaN                  NaN   
39932                         NaN                  NaN   
40019                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
39848                       NaN                         NaN   
39853                       NaN                         NaN   
39891                       NaN                         NaN   
39932                       NaN                         NaN   
40019                       NaN                         NaN   
39848        

In [22]:


df[(df['game_id'] == 2019120808) & (df['interception'] == 1)]
# play_type is no play

Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
34568       34569       NaN       NaN       11.0            0.0   
34638       34639 -0.699744 -0.021898        3.0            0.0   
34720       34721  2.628091  0.105108       24.0            0.0   
34638       34639 -0.699744 -0.021898        3.0            0.0   
34720       34721  2.628091  0.105108       24.0            0.0   

      assist_tackle_1_player_id assist_tackle_1_player_name  \
34568                       NaN                         NaN   
34638                       NaN                         NaN   
34720                       NaN                         NaN   
34638                       NaN                         NaN   
34720                       NaN                         NaN   

      assist_tackle_1_team assist_tackle_2_player_id  \
34568                  NaN                       NaN   
34638                  NaN                       NaN   
34720                  NaN                       NaN   
34638                  NaN                       NaN   
34720                  NaN                       NaN   

      assist_tackle_2_player_name assist_tackle_2_team  \
34568                         NaN                  NaN   
34638                         NaN                  NaN   
34720                         NaN                  NaN   
34638                         NaN                  NaN   
34720                         NaN                  NaN   

      assist_tackle_3_player_id assist_tackle_3_player_name  \
34568                       NaN                         NaN   
34638                       NaN                         NaN   
34720                       NaN                         NaN   
34638                       NaN                         NaN   
34720                       NaN                         NaN   

      assist_tackle_3_team assist_tackle_4_player_id  \
34568                  NaN                       NaN   
34638                  NaN                       NaN   
34720                  NaN                       NaN   
34638                  NaN                       NaN   
34720                  NaN                       NaN   

      assist_tackle_4_player_name assist_tackle_4_team away_team  \
34568                         NaN                  NaN       IND   
34638                         NaN                  NaN       IND   
34720                         NaN                  NaN       IND   
34638                         NaN                  NaN       IND   
34720                         NaN                  NaN       IND   

       away_timeouts_remaining   away_wp  away_wp_post blocked_player_id  \
34568                        3  0.469926      0.567726               NaN   
34638                        3  0.522383      0.812424               NaN   
34720                        3  0.702698      0.801927               NaN   
34638                        3  0.522383      0.812424               NaN   
34720                        3  0.702698      0.801927               NaN   

      blocked_player_name  comp_air_epa  comp_air_wpa  comp_yac_epa  \
34568                 NaN           0.0           0.0           0.0   
34638                 NaN           0.0           0.0           0.0   
34720                 NaN           0.0           0.0           0.0   
34638                 NaN           0.0           0.0           0.0   
34720                 NaN           0.0           0.0           0.0   

       comp_yac_wpa  complete_pass    def_wp  defensive_extra_point_attempt  \
34568           0.0            0.0  0.469926                            0.0   
34638           0.0            0.0  0.522383                            0.0   
34720           0.0            0.0  0.702698                            0.0   
34638           0.0            0.0  0.522383                            0.0   
34720           0.0            0.0  0.702698                            0.0   

       defensive_extra_point_conv  defensive_two_point_

In [15]:
df[(df['play_maker'] == 'M.Ryan') & (df['interception'] != 0)].shape

(12, 264)

In [ ]:
"""
RB_stats
"""

In [42]:
pd.set_option('display.max_rows',None)

RB_stats = df[df['play_type'] == 'run'].groupby('play_maker', as_index = False).agg({
    'yards_gained':['sum', 'max'],
    'rush_touchdown':['sum'],
    'fumble':['sum']
    
})

RB_stats.columns = ['play_maker', 'Yards', 'Long', 'rush_touchdown', 'fumble']

RB_stats

play_maker  Yards  Long  rush_touchdown  fumble
0            A.Abdullah    115    15             0.0     0.0
1               A.Armah     11     4             1.0     0.0
2                A.Beck      3     3             0.0     0.0
3               A.Brown     65    49             1.0     0.0
4              A.Cooper      6     6             0.0     0.0
5              A.Dalton     87    17             4.0     3.0
6              A.Ekeler    557    35             3.0     1.0
7            A.Erickson     33    17             0.0     0.0
8           A.Humphries      1     1             0.0     0.0
9              A.Ingold     17     4             0.0     0.0
10           A.Isabella     15     6             0.0     0.0
11           A.Janovich      1     1             1.0     0.0
12            A.Jeffery      2     2             1.0     0.0
13              A.Jones   1084    56            16.0     2.0
14             A.Kamara    797    40             5.0     3.0
15             A.Lazard     21    21             0.0     0.0
16             A.Levine     60    60             0.0     0.0
17         A.Levine Sr.      2     2             0.0     0.0
18           A.Mattison    462    35             1.0     1.0
19           A.McCarron     39    18             1.0     0.0
20             A.Miller     -1    -1             0.0     1.0
21             A.Morris      4     4             0.0     0.0
22           A.Peterson    898    32             5.0     2.0
23            A.Roberts      7     7             0.0     0.0
24           A.Robinson      2     2             0.0     0.0
25            A.Rodgers    204    17             1.0     1.0
26            A.Sherman      9     5             0.0     0.0
27            A.Thielen      6     3             1.0     0.0
28             A.Wilson     45    28             0.0     0.0
29              B.Allen     42    11             0.0     0.0
30              B.Anger     -5    -5             0.0     0.0
31             B.Bolden     68    21             3.0     0.0
32            B.Bortles     -9     1             0.0     0.0
33              B.Cooks     52    27             0.0     0.0
34             B.Fowler     20    20             0.0     0.0
35               B.Hill    323    27             2.0     0.0
36              B.Hoyer      7     6             0.0     0.0
37            B.Hundley     44    14             0.0     0.0
38               B.Kern      0     0             0.0     0.0
39           B.Mayfield    144    18             3.0     2.0
40           B.Perriman     16    13             0.0     0.0
41             B.Powell    229    17             0.0     0.0
42     B.Roethlisberger      7     7             0.0     0.0
43         B.Scarbrough    377    30             1.0     1.0
44              B.Scott    245    25             5.0     1.0
45              B.Snell    118    23             0.0     0.0
46          B.Snell Jr.    308    21             2.0     1.0
47           C.Anderson     43     9             0.0     0.0
48             C.Carson   1230    59             7.0     6.0
49             C.Daniel      6     4             0.0     0.0
50            C.Edmonds    303    37             4.0     0.0
51             C.Godwin      8     8             0.0     0.0
52                C.Ham     17     9             0.0     0.0
53               C.Hyde   1070    58             6.0     4.0
54              C.Jones      5     5             0.0     0.0
55             C.Keenum     15     9             1.0     2.0
56               C.Kirk     93    28             0.0     0.0
57               C.Kupp      4     6             0.0     0.0
58          C.McCaffrey   1389    84            15.0     1.0
59              C.McCoy     14     9             0.0     0.0
60              C.Moore      0     0             0.0     0.0
61             C.Newton     -2     3             0.0     2.0
62          C.Patterson    103    46             0.0     0.0
63            C.Prosise     72    17             1.0     1.0
64             C.Ridley     34    19            

In [56]:
df_RB = df_RB.rename(columns = {'Player': 'play_maker'})
valid_table_RB = RB_stats.merge(df_RB, on = 'play_maker', how = 'outer')
valid_table_RB

play_maker   Yards  Long  rush_touchdown  fumble     Rk   Tm  \
0            A.Abdullah   115.0  15.0             0.0     0.0  116.0  MIN   
1               A.Armah    11.0   4.0             1.0     0.0  177.0  CAR   
2                A.Beck     3.0   3.0             0.0     0.0  281.0  DEN   
3               A.Brown    65.0  49.0             1.0     0.0  222.0  TEN   
4               A.Brown    65.0  49.0             1.0     0.0  282.0  NWE   
5              A.Cooper     6.0   6.0             0.0     0.0  285.0  DAL   
6              A.Dalton    87.0  17.0             4.0     3.0  102.0  CIN   
7              A.Ekeler   557.0  35.0             3.0     1.0   35.0  LAC   
8            A.Erickson    33.0  17.0             0.0     0.0  186.0  CIN   
9           A.Humphries     1.0   1.0             0.0     0.0  301.0  TEN   
10             A.Ingold    17.0   4.0             0.0     0.0  155.0  OAK   
11           A.Isabella    15.0   6.0             0.0     0.0  210.0  ARI   
12           A.Janovich     1.0   1.0             1.0     0.0  302.0  DEN   
13            A.Jeffery     2.0   2.0             1.0     0.0  303.0  PHI   
14              A.Jones  1084.0  56.0            16.0     2.0   15.0  GNB   
15             A.Kamara   797.0  40.0             5.0     3.0   25.0  NOR   
16             A.Lazard    21.0  21.0             0.0     0.0  312.0  GNB   
17             A.Levine    60.0  60.0             0.0     0.0  262.0  BAL   
18         A.Levine Sr.     2.0   2.0             0.0     0.0    NaN  NaN   
19           A.Mattison   462.0  35.0             1.0     1.0   47.0  MIN   
20           A.McCarron    39.0  18.0             1.0     0.0  190.0  HOU   
21             A.Miller    -1.0  -1.0             0.0     1.0  316.0  CHI   
22             A.Morris     4.0   4.0             0.0     0.0  318.0  ARI   
23           A.Peterson   898.0  32.0             5.0     2.0   19.0  WAS   
24            A.Roberts     7.0   7.0             0.0     0.0  322.0  BUF   
25           A.Robinson     2.0   2.0             0.0     0.0  323.0  CHI   
26            A.Rodgers   204.0  17.0             1.0     1.0   84.0  GNB   
27            A.Sherman     9.0   5.0             0.0     0.0  216.0  KAN   
28            A.Thielen     6.0   3.0             1.0     0.0  218.0  MIN   
29             A.Wilson    45.0  28.0             0.0     0.0  198.0  MIA   
30              B.Allen    42.0  11.0             0.0     0.0  151.0  DEN   
31              B.Anger    -5.0  -5.0             0.0     0.0  280.0  HOU   
32             B.Bolden    68.0  21.0             3.0     0.0  139.0  NWE   
33            B.Bortles    -9.0   1.0             0.0     0.0  250.0  LAR   
34              B.Cooks    52.0  27.0             0.0     0.0  179.0  LAR   
35             B.Fowler    20.0  20.0             0.0     0.0  292.0  NYG   
36               B.Hill   323.0  27.0             2.0     0.0   54.0  ATL   
37              B.Hoyer     7.0   6.0             0.0     0.0  169.0  IND   
38            B.Hundley    44.0  14.0             0.0     0.0  174.0  ARI   
39               B.Kern     0.0   0.0             0.0     0.0  309.0  TEN   
40           B.Mayfield   144.0  18.0             3.0     2.0  109.0  CLE   
41           B.Perriman    16.0  13.0             0.0     0.0  269.0  TAM   
42             B.Powell   229.0  17.0             0.0     0.0   69.0  NYJ   
43     B.Roethlisberger     7.0   7.0             0.0     0.0  324.0  PIT   
44         B.Scarbrough   377.0  30.0             1.0     1.0   50.0  DET   
45              B.Scott   245.0  25.0             5.0     1.0   67.0  PHI   
46              B.Snell   118.0  23.0             0.0     0.0    NaN  NaN   
47          B.Snell Jr.   308.0  21.0             2.0     1.0    NaN  NaN   
48           C.Anderson    43.0   9.0             0.0     0.0  137.0  DET   
49             C.Carson  1230.0  59.0             7.0     6.0    5.0  SEA   
50             C.Daniel     6.0   4.0             0.0     0.0  180.0  CHI   
51       

In [ ]:
"""
WR_stats
"""

In [57]:
pd.set_option('display.max_rows',None)

WR_stats = df[df['play_type'] == 'reception'].groupby('play_maker', as_index = False).agg({
    'yards_gained':['sum', 'max'],
    'pass_touchdown':['sum'],
    'fumble':['sum'],
    'reception':['sum'],
    'Target':['sum']
    
})

WR_stats.columns = ['play_maker', 'Yards', 'Long', 'receiving_touchdown', 'fumble', 'reception', 'target']
WR_stats

play_maker  Yards  Long  receiving_touchdown  fumble  reception  \
0            A.Abdullah     88    16                  1.0     0.0         15   
1               A.Armah      6     4                  0.0     0.0          2   
2             A.Auclair     11    11                  0.0     0.0          1   
3                A.Beck     90    29                  1.0     0.0          9   
4               A.Brown   1107    91                  9.0     1.0         53   
5            A.Callaway     89    41                  0.0     0.0          7   
6                A.Carr      9     9                  0.0     0.0          1   
7              A.Cooper   1189    53                  8.0     0.0         78   
8              A.Dalton     -4    -4                  0.0     0.0          1   
9               A.Dulin     17    13                  0.0     0.0          2   
10             A.Ekeler    993    84                  8.0     2.0         92   
11           A.Erickson    529    52                  0.0     1.0         43   
12            A.Firkser    204    39                  1.0     0.0         14   
13            A.Hollins     46    35                  0.0     0.0          2   
14             A.Hooper    789    35                  6.0     0.0         75   
15          A.Humphries    374    30                  2.0     0.0         37   
16              A.Hurns    416    27                  2.0     1.0         32   
17             A.Ingold     44    14                  1.0     0.0          6   
18           A.Isabella    189    88                  1.0     0.0          9   
19           A.Janovich     42    22                  0.0     0.0          5   
20            A.Jeffery    490    38                  4.0     0.0         42   
21              A.Jones    474    67                  3.0     1.0         48   
22             A.Kamara    533    41                  1.0     1.0         81   
23             A.Lazard    477    43                  3.0     0.0         35   
24           A.Mattison     82    17                  0.0     0.0         10   
25             A.Miller    656    35                  2.0     0.0         52   
26             A.Patton     56    15                  0.0     0.0          6   
27           A.Peterson    142    22                  0.0     1.0         17   
28            A.Roberts     20     7                  0.0     0.0          3   
29           A.Robinson   1147    49                  7.0     0.0         96   
30            A.Shaheen     76    18                  0.0     0.0          9   
31            A.Sherman     22    15                  0.0     0.0          2   
32               A.Tate    575    33                  1.0     1.0         40   
33            A.Thielen    422    44                  6.0     1.0         29   
34             A.Wilson    351    35                  1.0     0.0         41   
35               B.Bell     67    30                  0.0     0.0          8   
36            B.Berrios    115    69                  0.0     0.0          5   
37             B.Bolden    111    29                  1.0     0.0          9   
38         B.Braunecker     59    18                  1.0     0.0          6   
39              B.Cooks    583    57                  2.0     0.0         41   
40             B.Fowler    195    17                  0.0     0.0         22   
41               B.Hill     69    12                  1.0     0.0         10   
42             B.Jarwin    365    42                  3.0     1.0         30   
43             B.Koyack      9     9                  0.0     0.0          1   
44           B.Perriman    645    44                  6.0     0.0         36   
45             B.Powell     33     9                  0.0     0.0          7   
46            B.Pringle    170    28                  1.0     0.0         12   
47         B.Scarbrough      5     5                  0.0     0.0          1   
48              B.Scott    204    39                  0.0     1.0         24   
49           

In [62]:
df_WR = df_WR.rename(columns = {'Player': 'play_maker'})
valid_table_WR = WR_stats.merge(df_WR, on = 'play_maker', how = 'outer')
valid_table_WR['delta_yds'] = valid_table_WR['Yards'] - valid_table_WR['Yds']
valid_table_WR[['play_maker', 'Yards', 'Yds', 'delta_yds']].sort_values(['delta_yds'], ascending = True)

# valid_table_WR

play_maker   Yards     Yds  delta_yds
109             D.Chark   528.0  1008.0     -480.0
266              J.Ross    28.0   506.0     -478.0
417             R.James    39.0   165.0     -126.0
352             M.Jones   655.0   779.0     -124.0
349            M.Ingram   125.0   247.0     -122.0
508             W.Snead   238.0   339.0     -101.0
360              M.Sanu   448.0   520.0      -72.0
391           P.Dorsett   347.0   397.0      -50.0
320             L.McCoy   158.0   181.0      -23.0
330            M.Ateman   107.0   116.0       -9.0
457           T.Edmunds    41.0    48.0       -7.0
204              G.Tate   670.0   676.0       -6.0
243            J.Holton    17.0    21.0       -4.0
436             S.Diggs  1126.0  1130.0       -4.0
317         L.Fournette   521.0   522.0       -1.0
332            M.Boykin   198.0   198.0        0.0
331             M.Boone    17.0    17.0        0.0
329           M.Andrews   852.0   852.0        0.0
337          M.Crabtree    22.0    22.0        0.0
333            M.Breida   120.0   120.0        0.0
0            A.Abdullah    88.0    88.0        0.0
326         L.Treadwell   184.0   184.0        0.0
327           L.Willson    79.0    79.0        0.0
338             M.Davis    22.0    22.0        0.0
325           L.Toilolo    10.0    10.0        0.0
324            L.Thomas   173.0   173.0        0.0
323           L.Stocker    53.0    53.0        0.0
322             L.Smith    31.0    31.0        0.0
321            L.Murray   235.0   235.0        0.0
319         L.Kendricks    50.0    50.0        0.0
316        L.Fitzgerald   804.0   804.0        0.0
328          M.Alie-Cox    93.0    93.0        0.0
339            M.Deiter     0.0     0.0        0.0
345            M.Gordon   296.0   296.0        0.0
342            M.Gaskin    51.0    51.0        0.0
373             N.Boyle   321.0   321.0        0.0
372           N.Bellore    23.0    23.0        0.0
371            N.Bawden    17.0    17.0        0.0
370           N.Agholor   363.0   363.0        0.0
367            M.Walton    89.0    89.0        0.0
366            M.Walker    15.0    15.0        0.0
365  M.Valdes-Scantling   452.0   452.0        0.0
364            M.Turner   245.0   245.0        0.0
359           M.Sanders   509.0   509.0        0.0
358            M.Pruitt    90.0    90.0        0.0
357              M.Mack    82.0    82.0        0.0
356             M.Lewis   156.0   156.0        0.0
355               M.Lee    18.0    18.0        0.0
354           M.LaCosse   131.0   131.0        0.0
351           M.Johnson   277.0   277.0        0.0
348           M.Hollins   125.0   125.0        0.0
347           M.Hardman   538.0   538.0        0.0
346              M.Hall   261.0   261.0        0.0
313            K.Stills   561.0   561.0        0.0
344           M.Goodwin   186.0   186.0        0.0
343           M.Gesicki   570.0   570.0        0.0
341            M.Gallup  1107.0  1107.0        0.0
309           K.Raymond   170.0   170.0        0.0
304              K.Hunt   285.0   285.0        0.0
307          K.Juszczyk   239.0   239.0        0.0
265           J.Richard   323.0   323.0        0.0
264          J.Reynolds   326.0   326.0        0.0
263           J.Perkins    87.0    87.0        0.0
262            J.Oliver    15.0    15.0        0.0
260     J.O'Shaughnessy   153.0   153.0        0.0
259            J.Nelson    36.0    36.0        0.0
258             J.Mundt    26.0    26.0        0.0
257             J.Moore    43.0    43.0        0.0
256             J.Mixon   287.0   287.0        0.0
255            J.Meyers   359.0   359.0        0.0
254          J.McKissic   233.0   233.0        0.0
252            J.Landry  1174.0  1174.0        0.0
251           J.Kumerow   219.0   219.0        0.0
250             J.Jones  1394.0  1394.0        0.0
249             J.James   142.0   142.0        0.0
248            J.Jacobs   166.0   166.0        0.0
247           J.Jackson    22.0    22.0        0.0
246            J.Howard    69.0    

In [46]:
new_df['Target'] = np.where(((new_df['play_type'] == 'pass') | (new_df['play_type'] == 'reception')) & (new_df['yards_after_catch'] != 'nan') & (new_df['sack'] == 0), 1, 0)


new_df['reception'] = np.where(((new_df['play_type'] == 'pass') | (new_df['play_type'] == 'reception')) & (new_df['yards_after_catch'] >= 0), 1, 0)


In [ ]:
"""
Testing
"""

In [65]:
All_M = new_df[(new_df['play_maker'] == 'M.Ryan') & (new_df['play_type'] == 'pass')]


# .groupby('game_id', as_index = False)

M_stats = All_M.groupby('game_id', as_index = False).agg({
    'yards_gained':['sum', 'max'],
    'pass_touchdown':['sum'],
    'interception':['sum'],
    'reception':['sum'],
    'Target':['sum'],
    'sack':['sum']
    
})

# M_sacks = All_M[All_M['sack'] == 1].groupby('game_id', as_index = False).agg({
#     'yards_gained':['sum'],
    
# })


# M.Stafford 8 games
#M.Ryan 15 games
# A.Rodgers 16 games


M_stats
# M_sacks
# g1 = All_M[All_M['game_id'] == 2019090804]
# g1[g1['sack'] == 1]

# possibly missing a game 

# yards in game 1 off by 8, missing 1 reception, 6 extra Targets
# yards in game 2 off by 6, correct
# up by 4 yards, one extra reception
# down by 15 yards, 5 extra Targets
# 4 yards down, reception down 1, Targets up 3
# yards down 8, Targets up 2
# Yards down 36, receptions are correct, Targets up 5

# we are missing receptions and thus the calculated yards are lower
# Yards gained factors in sacks, needs to be taken out
# QB_rush yards




game_id yards_gained     pass_touchdown interception reception Target  \
                        sum max            sum          sum       sum    sum   
0   2019090804          272  23            2.0          2.0        32     52   
1   2019091513          310  54            3.0          2.0        27     43   
2   2019092203          304  34            3.0          1.0        28     34   
3   2019092900          364  28            0.0          0.0        35     58   
4   2019100602          318  29            3.0          1.0        31     49   
5   2019101307          341  30            4.0          0.0        30     38   
6   2019102000          121  39            0.0          1.0        16     32   
7   2019111004          174  54            2.0          1.0        19     36   
8   2019111701          292  48            1.0          0.0        21     35   
9   2019112400          225  53            0.0          1.0        22     52   
10  2019112802          259  23            2.0          1.0        32     58   
11  2019120800          302  93            2.0          0.0        19     35   
12  2019121512          201  28            2.0          0.0        25     41   
13  2019122205          383  32            1.0          2.0        32     46   
14  2019122912          286  35            1.0          0.0        30     57   

   sack  
    sum  
0   4.0  
1   1.0  
2   0.0  
3   5.0  
4   2.0  
5   2.0  
6   5.0  
7   1.0  
8   3.0  
9   6.0  
10  9.0  
11  1.0  
12  2.0  
13  1.0  
14  6.0

In [66]:
M_sacks = All_M[All_M['sack'] == 1].groupby('game_id', as_index = False).agg({
    'yards_gained':['sum'],
    
})

M_sacks
g1 = All_M[All_M['game_id'] == 2019091513]
g1[g1['sack'] == 1]


Unnamed: 0  air_epa  air_wpa  air_yards  assist_tackle  \
5476        5477      NaN      NaN        NaN            0.0   

     assist_tackle_1_player_id assist_tackle_1_player_name  \
5476                       NaN                         NaN   

     assist_tackle_1_team assist_tackle_2_player_id  \
5476                  NaN                       NaN   

     assist_tackle_2_player_name assist_tackle_2_team  \
5476                         NaN                  NaN   

     assist_tackle_3_player_id assist_tackle_3_player_name  \
5476                       NaN                         NaN   

     assist_tackle_3_team assist_tackle_4_player_id  \
5476                  NaN                       NaN   

     assist_tackle_4_player_name assist_tackle_4_team away_team  \
5476                         NaN                  NaN       PHI   

      away_timeouts_remaining   away_wp  away_wp_post blocked_player_id  \
5476                        3  0.279839      0.380341               NaN   

     blocked_player_name  comp_air_epa  comp_air_wpa  comp_yac_epa  \
5476                 NaN           0.0           0.0           0.0   

      comp_yac_wpa  complete_pass    def_wp  defensive_extra_point_attempt  \
5476           0.0            0.0  0.279839                            0.0   

      defensive_extra_point_conv  defensive_two_point_attempt  \
5476                         0.0                          0.0   

      defensive_two_point_conv defteam  defteam_score  defteam_score_post  \
5476                       0.0     PHI           12.0                12.0   

      defteam_timeouts_remaining  \
5476                         3.0   

                                                   desc  down  drive  \
5476  (12:20) (Shotgun) M.Ryan sacked at ATL 36 for ...   3.0     20   

            ep      epa  extra_point_attempt  extra_point_prob  \
5476  1.300972 -2.54673                  0.0               0.0   

     extra_point_result   fg_prob  field_goal_attempt field_goal_result  \
5476                NaN  0.292009                 0.0               NaN   

      first_down_pass  first_down_penalty  first_down_rush  \
5476              0.0                 0.0              0.0   

     forced_fumble_player_1_player_id forced_fumble_player_1_player_name  \
5476                              NaN                                NaN   

     forced_fumble_player_1_team forced_fumble_player_2_player_id  \
5476                         NaN                              NaN   

     forced_fumble_player_2_player_name forced_fumble_player_2_team  \
5476                                NaN                         NaN   

      fourth_down_converted  fourth_down_failed  fumble  fumble_forced  \
5476                    0.0                 0.0     0.0            0.0   

      fumble_lost  fumble_not_forced  fumble_out_of_bounds  \
5476          0.0                0.0                   0.0   

     fumble_recovery_1_player_id fumble_recovery_1_player_name  \
5476                         NaN                           NaN   

     fumble_recovery_1_team  fumble_recovery_1_yards  \
5476                    NaN                      NaN   

     fumble_recovery_2_player_id fumble_recovery_2_player_name  \
5476                         NaN                           NaN   

     fumble_recovery_2_team  fumble_recovery_2_yards fumbled_1_player_id  \
5476                    NaN                      NaN                 NaN   

     fumbled_1_player_name fumbled_1_team fumbled_2_player_id  \
5476                   NaN            NaN                 NaN   

     fumbled_2_player_name fumbled_2_team   game_date game_half     game_id  \
5476                   NaN            NaN  2019-09-15     Half2  2019091513   

      game_seconds_remaining  goal_to_go  half_seconds_remaining home_team  \
5476                   740.0         0.0                   740.0       ATL   

      home_timeouts_remaining   home_wp  home_wp_post  incomplete_pass  \
5476                        3  

In [61]:
Matty_ice = new_df[new_df['play_maker'] == 'M.Ryan']

# Matty_ice[Matty_ice['game_id'] == '2019091513']

M_run = Matty_ice[Matty_ice['play_type'] == 'run']
M_run[M_run['game_id'] == 2019090804]





Unnamed: 0  air_epa  air_wpa  air_yards  assist_tackle  \
735         736      NaN      NaN        NaN            0.0   
777         778      NaN      NaN        NaN            0.0   

    assist_tackle_1_player_id assist_tackle_1_player_name  \
735                       NaN                         NaN   
777                       NaN                         NaN   

    assist_tackle_1_team assist_tackle_2_player_id  \
735                  NaN                       NaN   
777                  NaN                       NaN   

    assist_tackle_2_player_name assist_tackle_2_team  \
735                         NaN                  NaN   
777                         NaN                  NaN   

    assist_tackle_3_player_id assist_tackle_3_player_name  \
735                       NaN                         NaN   
777                       NaN                         NaN   

    assist_tackle_3_team assist_tackle_4_player_id  \
735                  NaN                       NaN   
777                  NaN                       NaN   

    assist_tackle_4_player_name assist_tackle_4_team away_team  \
735                         NaN                  NaN       ATL   
777                         NaN                  NaN       ATL   

     away_timeouts_remaining   away_wp  away_wp_post blocked_player_id  \
735                        3  0.436599      0.447745               NaN   
777                        3  0.166709      0.178541               NaN   

    blocked_player_name  comp_air_epa  comp_air_wpa  comp_yac_epa  \
735                 NaN           0.0           0.0           0.0   
777                 NaN           0.0           0.0           0.0   

     comp_yac_wpa  complete_pass    def_wp  defensive_extra_point_attempt  \
735           0.0            0.0  0.563401                            0.0   
777           0.0            0.0  0.833291                            0.0   

     defensive_extra_point_conv  defensive_two_point_attempt  \
735                         0.0                          0.0   
777                         0.0                          0.0   

     defensive_two_point_conv defteam  defteam_score  defteam_score_post  \
735                       0.0     MIN            0.0                 0.0   
777                       0.0     MIN           14.0                14.0   

     defteam_timeouts_remaining  \
735                         3.0   
777                         3.0   

                                                  desc  down  drive        ep  \
735  (13:41) (Shotgun) M.Ryan scrambles left end to...   3.0      1 -1.504026   
777  (11:26) M.Ryan scrambles right end pushed ob a...   1.0      7  3.670438   

          epa  extra_point_attempt  extra_point_prob extra_point_result  \
735  0.301238                  0.0               0.0                NaN   
777  0.502483                  0.0               0.0                NaN   

      fg_prob  field_goal_attempt field_goal_result  first_down_pass  \
735  0.150757                 0.0               NaN              0.0   
777  0.332221                 0.0               NaN              0.0   

     first_down_penalty  first_down_rush forced_fumble_player_1_player_id  \
735                 0.0              0.0                              NaN   
777                 0.0              1.0                              NaN   

    forced_fumble_player_1_player_name forced_fumble_player_1_team  \
735                                NaN                         NaN   
777                                NaN                         NaN   

    forced_fumble_player_2_player_id forced_fumble_player_2_player_name  \
735                              NaN                                NaN   
777                              NaN                                NaN   

    forced_fumble_player_2_team  fourth_down_converted  fourth_down_failed  \
735                         NaN                    0.0                 0.0   
777                         NaN                   

In [49]:
MS_stats = new_df[new_df['play_maker'] == 'M.Stafford'].groupby('game_id', as_index = False).agg({
    'yards_gained':['sum', 'max'],
    'pass_touchdown':['sum'],
    'interception':['sum'],
    'reception':['sum'],
    'Target':['sum']
    
})

MS_stats

# 2 yards short, 1 target extra
# 14 yards short, 1 missing reception
# last game in off by 17 yards but completions are good



game_id yards_gained     pass_touchdown interception reception Target
                       sum max            sum          sum       sum    sum
0  2019090809          383  47            3.0          0.0        27     46
1  2019091502          259  36            2.0          2.0        21     30
2  2019092207          204  27            1.0          0.0        18     32
3  2019092903          279  34            3.0          0.0        21     38
4  2019101400          253  66            0.0          0.0        18     35
5  2019102003          352  47            4.0          1.0        30     48
6  2019102703          316  49            3.0          1.0        24     36
7  2019110307          389  59            3.0          1.0        26     43

In [55]:
game2 = new_df[new_df['game_id'] == 2019091502]
game2[game2['defteam'] != 'DET']
# game2
# new_df[new_df['play_maker'] == 'M.Stafford']

Unnamed: 0   air_epa   air_wpa  air_yards  assist_tackle  \
4121        4122       NaN       NaN        NaN            1.0   
4122        4123       NaN       NaN        NaN            1.0   
4123        4124  2.700473  0.079952       10.0            0.0   
4124        4125       NaN       NaN        NaN            0.0   
4136        4137       NaN       NaN        NaN            0.0   
4137        4138  0.548376  0.019347        7.0            0.0   
4138        4139       NaN       NaN        NaN            1.0   
4139        4140 -0.889961 -0.030462       -2.0            0.0   
4140        4141  0.997872  0.033916       11.0            0.0   
4141        4142       NaN       NaN        NaN            1.0   
4142        4143       NaN       NaN        NaN            1.0   
4143        4144  0.816024  0.026293       24.0            0.0   
4144        4145 -0.932588 -0.032341       -2.0            0.0   
4145        4146       NaN       NaN        NaN            0.0   
4151        4152       NaN       NaN        NaN            0.0   
4152        4153       NaN       NaN        NaN            0.0   
4153        4154       NaN       NaN        NaN            0.0   
4154        4155       NaN       NaN        NaN            NaN   
4155        4156 -1.053975 -0.032133       -1.0            0.0   
4156        4157  2.689365  0.080521       12.0            0.0   
4157        4158 -0.398255 -0.012022        3.0            0.0   
4158        4159  0.095810  0.003214        6.0            0.0   
4159        4160       NaN       NaN        NaN            0.0   
4160        4161       NaN       NaN        NaN            0.0   
4165        4166       NaN       NaN        NaN            NaN   
4167        4168       NaN       NaN        NaN            1.0   
4168        4169       NaN       NaN        NaN            0.0   
4169        4170  1.396539  0.045653       11.0            0.0   
4170        4171       NaN       NaN        NaN            0.0   
4171        4172       NaN       NaN        NaN            1.0   
4172        4173       NaN       NaN        NaN            0.0   
4173        4174 -0.356109 -0.011257        2.0            1.0   
4174        4175       NaN       NaN        NaN            0.0   
4175        4176  1.370567  0.045702        6.0            0.0   
4176        4177       NaN       NaN        NaN            0.0   
4183        4184 -0.195737 -0.006266        4.0            0.0   
4184        4185       NaN       NaN        NaN            0.0   
4185        4186       NaN       NaN        NaN            0.0   
4186        4187  2.081830  0.064348       22.0            0.0   
4187        4188       NaN       NaN        NaN            NaN   
4188        4189       NaN       NaN        NaN            0.0   
4189        4190  0.565880  0.017749        6.0            0.0   
4190        4191  1.073068  0.033588        9.0            0.0   
4191        4192       NaN       NaN        NaN            0.0   
4192        4193       NaN       NaN        NaN            0.0   
4202        4203       NaN       NaN        NaN            NaN   
4203        4204       NaN       NaN        NaN            0.0   
4204        4205       NaN       NaN        NaN            0.0   
4205        4206 -0.245246 -0.008794        5.0            0.0   
4206        4207 -0.061339 -0.003517        6.0            0.0   
4207        4208       NaN       NaN        NaN            1.0   
4208        4209       NaN       NaN        NaN            1.0   
4209        4210  2.209764  0.070961       25.0            0.0   
4210        4211       NaN       NaN        NaN            0.0   
4211        4212  3.406569  0.110715       36.0            0.0   
4221        4222 -0.363674 -0.012819        4.0            0.0   
4222        4223       NaN       NaN        NaN            1.0   
4223        4224       NaN       NaN        NaN            1.0   
4224        4225       NaN       NaN        NaN            1.0   
4233        4234  0.013277  0.000304        8.